In [ ]:
!pip uninstall torchaudio

Found existing installation: torchaudio 0.10.0+cu113
Uninstalling torchaudio-0.10.0+cu113:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchaudio-0.10.0+cu113.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchaudio/*
Proceed (y/n)? y
  Successfully uninstalled torchaudio-0.10.0+cu113


In [ ]:
!wget https://ai-data.obs.ru-moscow-1.hc.sbercloud.ru/МО_на_графах.zip
!unzip МО_на_графах.zip
!unzip /content/ВК/hackathon_mos_sep2022.zip -d ./zippeddataset/
!unzip /content/zippeddataset/FINAL_ALL_SEQUENCES_TRAINTEST.tsv.zip -d ./dataset/
!unzip /content/zippeddataset/FINAL_FEATURES_FRIENDS.tsv.zip -d ./dataset/
!unzip /content/zippeddataset/FINAL_FEATURES_TRAINTEST.tsv.zip -d ./dataset/
!unzip /content/zippeddataset/FINAL_SEQUENCES_MATRIX.tsv.zip -d ./dataset/
!unzip /content/zippeddataset/FINAL_TARGETS_DATES_TRAINTEST.tsv.zip -d ./dataset/

--2022-09-25 02:44:17--  https://ai-data.obs.ru-moscow-1.hc.sbercloud.ru/%D0%9C%D0%9E_%D0%BD%D0%B0_%D0%B3%D1%80%D0%B0%D1%84%D0%B0%D1%85.zip
Resolving ai-data.obs.ru-moscow-1.hc.sbercloud.ru (ai-data.obs.ru-moscow-1.hc.sbercloud.ru)... 46.243.206.34, 46.243.206.35
Connecting to ai-data.obs.ru-moscow-1.hc.sbercloud.ru (ai-data.obs.ru-moscow-1.hc.sbercloud.ru)|46.243.206.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3399326506 (3.2G) [application/zip]
Saving to: ‘МО_на_графах.zip’

МО_на_графах.zip    100%[===================>]   3.17G  69.9MB/s    in 48s     

2022-09-25 02:45:05 (66.9 MB/s) - ‘МО_на_графах.zip’ saved [3399326506/3399326506]

Archive:  МО_на_графах.zip
  inflating: ВК/hackathon_mos_sep2022.zip  
error: invalid zip file with overlapped components (possible zip bomb)
Archive:  /content/ВК/hackathon_mos_sep2022.zip
 extracting: ./zippeddataset/FINAL_ALL_SEQUENCES_TRAINTEST.tsv.zip  
 extracting: ./zippeddataset/FINAL_FEATURES_FRIENDS.tsv.zip  

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 7.9 MB 14.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 12.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 10.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=fbdfebef8f282d50c05cb35d9899fd12bfba2e930944da1b0624f540ee17afc8
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


In [ ]:
import os.path as osp

import numpy as np
import pandas as pd
import csv
import torch
from torch_geometric.data import Dataset, download_url, Data


class NodeDataset(Dataset):
    def __init__(self, count_rows = None, root = "/content/dataset/", transform=None, pre_transform=None, pre_filter=None):
        self.sequence_predict = []
        self.files_data = []
        self.count_rows = count_rows
        
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']

    @property
    def processed_file_names(self):
        return self.files_data
    
    def download(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']
    
    def process_sequence(self, sequence, latent_space = 50):
        processed_sequence = torch.from_numpy(np.zeros((latent_space)))
        series = pd.rear_csv('dict_hashes.csv')
        dict_hashes = series.to_dict()
        series = pd.rear_csv('rand_v.csv')
        rand_v = series.to_dict()

        processed_vec = [[rand_v[dict_hashes[j]][0] for j in row] for row in a.values()]
        token_start = 10
        token_end = -10
        tokenized_vec = [[token_start] + i + [token_end] for i in processed_vec]

        latent_length = 100

        post_processed = np.array([np.concatenate((np.zeros((latent_length)), i))[-latent_length:] for i in tokenized_vec])
        PATH = ''
        lstm = torch.load(PATH)
        with torch.no_grad():
          tag_scores = lstm(torch.from_numpy(post_processed).float())
        self.sequence_predict.append(tag_scores[1])
        
    
    def process(self):
        # последовательности пользовательских состояний для пользователей из основной выборки
        ALL_SEQUENCES_TRAINTEST = self.root + '/FINAL_ALL_SEQUENCES_TRAINTEST.tsv'

        # связь между пользователем из основной выборки и всеми его друзьями (граф) + те же самые два домена признаков для друзей
        FEATURES_FRIENDS = self.root + '/FINAL_FEATURES_FRIENDS.tsv'

        # 2 домена вещественных признаков
        FEATURES_TRAINTEST = self.root + '/FINAL_FEATURES_TRAINTEST.tsv'

        # метки классов, ретродаты и идентификаторы тестовых объектов
        TARGETS_DATES_TRAINTEST = self.root + '/FINAL_TARGETS_DATES_TRAINTEST.tsv'
        
        ####DATAFRAMES####
        
        client_sequences = pd.read_csv(ALL_SEQUENCES_TRAINTEST,sep='\t')
        
        targets = pd.read_csv(TARGETS_DATES_TRAINTEST,sep='\t')
        
        features = pd.read_csv(FEATURES_TRAINTEST,sep='\t')
        
        ####Process graph####
                
        client_set = set()    
        
        
        dataframe_friends = pd.DataFrame()
        chunksize = 100
        rows = 0
        idx = 0
        for friends_chunk in pd.read_csv(FEATURES_FRIENDS, sep='\t', chunksize=chunksize, nrows=self.count_rows):
            for row in friends_chunk.iloc():
                rows += 1;
                if rows % 10000 == 0:
                    if self.count_rows == None: print(rows)
                    else:  print(float(rows) / float(self.count_rows) * 100, "%")
                if row['CLIENT_ID'] in client_set:
                    dataframe_friends = pd.concat([dataframe_friends, row], axis=1)
                else:
                    if len(client_set) > 0:
                        client_feature = torch.tensor([features[features['CLIENT_ID'] == row['CLIENT_ID']].values[0][1:]])
                        friend_feature = torch.tensor(dataframe_friends.values)[1:-1]
                        
                        if len(friend_feature.shape) == 1: friend_feature = friend_feature.reshape(1014, -1)
                        
                        friend_feature = friend_feature.transpose(0, 1)
                        
                        X = torch.cat((client_feature, friend_feature))
                        
                        edges = torch.from_numpy(np.concatenate((np.zeros((len(friend_feature))), np.arange(1,len(friend_feature) + 1))).reshape((2, -1))).int()
                        edge_attrs = torch.tensor(dataframe_friends.values)[-1] % 100
                                                
                        sequence = client_sequences[client_sequences['CLIENT_ID'] == row['CLIENT_ID']]['SEQUENCE'].values
                        self.process_sequence(sequence)
                        
                        predict = targets[targets['CLIENT_ID'] == row['CLIENT_ID']]['TARGET'].values
                        
                        if predict[0] == 'test': predict[0] = -1
                        else: predict[0] = float(predict[0])
                        
                        predict = torch.from_numpy(np.array(predict).astype(float))
                        
                        data = Data(
                            x = X,
                            edge_index = edges,
                            edge_attr = edge_attrs,
                            y = predict
                        )
                        
                        torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                        self.files_data.append(osp.join(self.processed_dir, f'data_{idx}.pt'))
                        idx += 1
                        
                    client_set.add(row['CLIENT_ID'])
                    dataframe_friends = row
                    
            

    def len(self):
        return len(self.files_data)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data
    
ds = NodeDataset(500)

Processing...


In [ ]:
ds[0]

Data(x=[81, 1014], edge_index=[2, 80], edge_attr=[80], y=[1])

In [ ]:
#%pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
torch.manual_seed(1)
torch.__version__

'1.12.1+cu113'

In [ ]:
data = pd.read_csv("/content/dataset/FINAL_SEQUENCES_MATRIX.tsv", sep='\t')

In [ ]:
a = {}

set_of_hash = set()

for index, row in data.iterrows():
    temp = [i[1:-1] for i in row['SESSIONS_SEQUENCES'][1:-1].split(", ")]

    for i in temp: 
      set_of_hash.add(i)

    a[index] = temp

n_latent = 1

rand_v = np.random.randn(len(set_of_hash), n_latent)

dict_hashes = {}

list_of_hash= list(set_of_hash)

for i, j in zip(range(len(set_of_hash)), list_of_hash):
   dict_hashes[j] = i

In [ ]:
# processed_vec = [[rand_v[dict_hashes[j]][0] for j in row] for row in a.values()]

In [ ]:
list(rand_v)

In [ ]:
# series = pd.Series(dict_hashes)
# series.to_csv('dict_hashes.csv', index=False)
# series = pd.Series(list(rand_v))
# series.to_csv('rand_v.csv', index=False)


In [ ]:
# series = pd.rear_csv('dict_hashes.csv')
# dict_hashes = series.to_dict()
# series = pd.rear_csv('rand_v.csv')
# rand_v = series.to_dict()

processed_vec = [[rand_v[dict_hashes[j]][0] for j in row] for row in a.values()]
token_start = 10
token_end = -10
tokenized_vec = [[token_start] + i + [token_end] for i in processed_vec]

latent_length = 100

post_processed = np.array([np.concatenate((np.zeros((latent_length)), i))[-latent_length:] for i in tokenized_vec])

In [ ]:
del data
del processed_vec
del tokenized_vec

In [ ]:
train = data_utils.TensorDataset(torch.from_numpy(post_processed).float())
train_loader = data_utils.DataLoader(train, batch_size=34, shuffle=True)

In [ ]:

class CreditsRNN(nn.Module):
    def __init__(self, seq_vec_size, rnn_units=2, top_classifier_units=2, num_out = 100):
        super(CreditsRNN, self).__init__()
        self.seq_vec_size = seq_vec_size
                        
        self._gru = nn.GRU(input_size=seq_vec_size,
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        
        self._hidden_size = rnn_units

        # self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self._top_classifier = nn.Linear(in_features=rnn_units, out_features=100)
        # self._intermediate_activation = nn.ReLU()
        # self._head = nn.Linear(in_features=top_classifier_units, out_features=num_out)
    
    def forward(self, seq_vec):

        
        _, last_hidden = self._gru(seq_vec)
                                
        classification_hidden = self._top_classifier(last_hidden)
        # activation = self._intermediate_activation(classification_hidden)
        # raw_output = self._head(activation)
        return classification_hidden, last_hidden
    

In [ ]:
# RNN Длина последовательности (seq_vec_size), латентное представление(rnn_units)
# Liner classification_hidden - выход, rnn_units - вход
model = CreditsRNN(seq_vec_size = 500, rnn_units = 50, num_out = 100)
model

CreditsRNN(
  (_gru): GRU(100, 50, batch_first=True)
  (_top_classifier): Linear(in_features=50, out_features=100, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
loss_function = torch.nn.L1Loss()
best_loss = 9999999
for epoch in range(500):  # again, normally you would NOT do 300 epochs, it is toy data
    norm = 0
    mae = 0
    val_data = []
    val_error = 0
    for x_batch in train_loader:
        norm +=1
        model.zero_grad()
        sequense, val = train_test_split(x_batch[0], test_size=0.1, random_state=42)
        val_data.append(val)
        tag_scores = model(sequense)
        loss = loss_function(tag_scores[0], sequense)
        mae += loss.tolist()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
       for val in val_data:
        tag_scores = model(val)
        loss = loss_function(tag_scores[0], val)
        val_error += loss.tolist()
    if best_loss<val_error/norm:
      best_loss = val_error/norm
      torch.save(model.state_dict(), '/content/sample_data/model_dict')
      torch.save(model, '/content/sample_data/model')
    
    print('Epoha=', epoch+1,' Train loss=', mae/norm, ' Val loss=', val_error/norm)

In [ ]:
class CreditsLSTM(nn.Module):
    def __init__(self, seq_vec_size, rnn_units, top_classifier_units=100, num_out = 100):
        super(CreditsLSTM, self).__init__()
        self.seq_vec_size = seq_vec_size

        self.lstm = nn.LSTM(input_size=seq_vec_size,
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
                        
        # self._gru = nn.GRU(input_size=seq_vec_size,
        #                      hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        
        self._hidden_size = rnn_units

        # self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self._top_classifier = nn.Linear(in_features=rnn_units, out_features=100)
        # self._intermediate_activation = nn.ReLU()
        # self._head = nn.Linear(in_features=top_classifier_units, out_features=num_out)
    
    def forward(self, seq_vec):

        
        _, last_hidden = self.lstm(seq_vec)
        # print(last_hidden)                                
        classification_hidden = self._top_classifier(last_hidden[0])
        # activation = self._intermediate_activation(classification_hidden)
        # raw_output = self._head(activation)
        return classification_hidden, last_hidden

In [ ]:
lstm = CreditsLSTM(seq_vec_size = 100, rnn_units = 50, num_out = 100)
lstm

CreditsLSTM(
  (lstm): LSTM(100, 50, batch_first=True)
  (_top_classifier): Linear(in_features=50, out_features=100, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(lr=1e-3, params=lstm.parameters())
loss_function = torch.nn.L1Loss()
best_loss = 9999999
for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    norm = 0
    mae = 0
    val_data = []
    val_error = 0
    for x_batch in train_loader:
        norm +=1
        lstm.zero_grad()
        sequense, val = train_test_split(x_batch[0], test_size=0.1, random_state=42)
        val_data.append(val)
        tag_scores = lstm(sequense)
        loss = loss_function(tag_scores[0], sequense)
        mae += loss.tolist()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
       for val in val_data:
        tag_scores = lstm(val)
        loss = loss_function(tag_scores[0], val)
        val_error += loss.tolist()
    if best_loss>val_error/norm:
      best_loss = val_error/norm
      torch.save(lstm.state_dict(), '/content/sample_data/lstm_dict.pth')
      torch.save(lstm, '/content/sample_data/lstm.pth')
    
    print('Epoha=', epoch+1,' Train loss=', mae/norm, ' Val loss=', val_error/norm)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([30, 100])) that is different to the input size (torch.Size([1, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([4, 100])) that is different to the input size (torch.Size([1, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoha= 1  Train loss= 0.23622847775909375  Val loss= 0.23612689835044154
Epoha= 2  Train loss= 0.23488073586930555  Val loss= 0.2495300196875443


In [ ]:
torch.save(lstm.state_dict(), '/content/sample_data/lstm_dict.pth')
torch.save(lstm, '/content/sample_data/lstm.pth')

In [ ]:
# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
with torch.no_grad():
    inputs = torch.Tensor([0,1,0,1,0])
    print(inputs.size(-1) )

    tag_scores = model(inputs)
    print(tag_scores)
    print(targets.shape)
    loss_function(tag_scores, targets)

In [ ]:
def eval_model(model: torch.nn.Module, dataset_val: List[str], batch_size: int = 32, device: torch.device = None) -> float:

    model.eval()
    preds = []
    targets = []
    val_generator = batches_generator(dataset_val, batch_size=batch_size, shuffle=False,
                                      device=device, is_train=True, output_format="torch")

    for batch in tqdm_notebook(val_generator, desc="Evaluating model"):
        targets.extend(batch["label"].detach().cpu().numpy().flatten())
        output = model(batch["features"])
        preds.extend(output.detach().cpu().numpy().flatten())

    return roc_auc_score(targets, preds)

In [ ]:
# Define the pytorch model
class torchLSTM(torch.nn.Module):
    def __init__(self, n_features, seq_length):
        super(torchLSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 100  # number of hidden states
        self.n_layers = 1  # number of LSTM layers (stacked)

        self.l_lstm = torch.nn.LSTM(input_size=n_features,
                                    hidden_size=self.n_hidden,
                                    num_layers=self.n_layers,
                                    batch_first=True)
        # according to pytorch docs LSTM output is
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden * self.seq_len, 3)

    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers, batch_size, self.n_hidden)
        cell_state = torch.zeros(self.n_layers, batch_size, self.n_hidden)
        self.hidden = (hidden_state, cell_state)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        lstm_out, self.hidden = self.l_lstm(x, self.hidden)
        # lstm_out(with batch_first = True) is
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size, -1)
        return self.l_linear(x)

In [ ]:
model_test = torchLSTM(5,5)
model_test

torchLSTM(
  (l_lstm): LSTM(5, 100, batch_first=True)
  (l_linear): Linear(in_features=500, out_features=3, bias=True)
)

In [ ]:
inputs = torch.Tensor([[[0,1,0,1,0,1]],[[0,1,0,1,0,1]]])

In [ ]:
inputs[0,0]

tensor([0., 1., 0., 1., 0., 1.])

In [ ]:
df = pd.DataFrame(data={'sequence': [[0,1,0,1,0,1],[0,1,0,1,0,1],[0,1,0,1,0,1],[0,1,0,1,0,1]]})

In [ ]:
def seq_data(df, num_pred):
  df['sequence']
  df['train'] = df['sequence'].apply(lambda x: x[:-num_pred])
  df['target'] = df['sequence'].apply(lambda x: x[-num_pred:])
  return df
seq_data(df, 1)

,sequence,train,target
0,"[0, 1, 0, 1, 0, 1]","[0, 1, 0, 1, 0]",[1]
1,"[0, 1, 0, 1, 0, 1]","[0, 1, 0, 1, 0]",[1]
2,"[0, 1, 0, 1, 0, 1]","[0, 1, 0, 1, 0]",[1]
3,"[0, 1, 0, 1, 0, 1]","[0, 1, 0, 1, 0]",[1]


In [ ]:
def norm_vector(vec, target):
  # print(target)
  len_vec = len(vec)
  target = vec[-1]
  vec = torch.Tensor(vec)
  vec = np.reshape(vec, (1,len_vec))
  target = torch.Tensor([target])
  # print(target)
  target = np.reshape(target, (1,1))
  # print(target)
  return vec, target



In [ ]:
los = torch.nn.L1Loss()
los(torch.Tensor([1,97,6,6]),torch.Tensor([2,9,8,]))

In [ ]:
for index, row in df.iterrows():
  vec, target = norm_vector(row['train'], row['target'])


In [ ]:
vec = [0,1,0,1,0,1]
len_vec = len(vec)
target = vec[-1]
b = torch.Tensor(b)
b = np.reshape(b, (1,6))
target = torch.Tensor(target)
target = np.reshape(target, (1,6))

tensor([0., 1., 0., 1., 0., 1.])

In [ ]:
def train_epoch(model: torch.nn.Module, optimizer: torch.optim.Optimizer,
                batch_size: int = 64, print_loss_every_n_batches: int = 500):
  
    model.train()
    loss_function = nn.BCEWithLogitsLoss(reduction="none")
    losses = torch.LongTensor()
    samples_counter = 0
    train_generator = batches_generator(dataset_train, batch_size=batch_size, shuffle=shuffle,
                                        device=device, is_train=True, output_format="torch")

    for num_batch, batch in tqdm_notebook(enumerate(train_generator, start=1), desc="Training"):
        output = torch.flatten(model(batch["features"]))
        batch_loss = loss_function(output, batch["label"].float())
        batch_loss.mean().backward()
        optimizer.step()
        optimizer.zero_grad()

        samples_counter += batch_loss.size(0)

        losses = torch.cat([losses, batch_loss], dim=0)
        if num_batch % print_loss_every_n_batches == 0:
            print(f"Batches {num_batch - print_loss_every_n_batches + 1} - {num_batch} loss:"
                  f"{losses[-samples_counter:].mean()}", end="\r")
            samples_counter = 0

    print(f"Training loss after epoch: {losses.mean()}", end="\r")


In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, size, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()

        self.hidden_dim = hidden_dim
        self.tagset_size = tagset_size
        # self.vec_seq = vec_seq
        self.size = size

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(1, size, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(5, 1) # количество разных состояний

    def forward(self, vec_seq):
        # vec_seq = self.vec_seq
        lstm_out, _ = self.lstm(vec_seq.view(self.size, 1, -1))
        # print(lstm_out.view(self.size, -1))
        tag_space = self.hidden2tag(lstm_out.view(self.size, -1))
        print(tag_space.shape)
        tag_scores = F.log_softmax(tag_space, dim=1)
        print(tag_scores.shape)
        return tag_scores

In [ ]:
model = LSTMTagger(size=5, hidden_dim=3, tagset_size=1)
model

LSTMTagger(
  (lstm): LSTM(1, 5, num_layers=3)
  (hidden2tag): Linear(in_features=5, out_features=1, bias=True)
)